In [1]:
# Utils for doing some logging on the results of the datasets (to be used later)

import json
from pathlib import Path


def get_unique_path(base_path: Path) -> Path:
    # return a unique file name
    if not base_path.exists():
        return base_path
    i = 1
    while True:
        new_path = base_path.with_name(f"{base_path.stem}_{i}{base_path.suffix}")
        if not new_path.exists():
            return new_path
        i += 1

def logging(results, dataset_name):
  # Save results to file
  output_dir = Path("outputs")
  output_dir.mkdir(parents=True, exist_ok=True)

  results_path = get_unique_path(output_dir / f"{dataset_name}_results.json")
  summary_path = get_unique_path(output_dir / f"{dataset_name}_summary.json")

  with open(results_path, "w") as f:
      json.dump(results, f, indent=2)

  # Compute and print accuracy
  correct = sum(r["reward"] for r in results if r["reward"] is not None)
  total = len(results)
  accuracy = correct / total if total > 0 else 0.0
  # Save summary to JSON

  summary = {
      "correct": correct,
      "total": total,
      "accuracy": accuracy
  }

  with open(summary_path, "w") as f:
      json.dump(summary, f, indent=2)
  print(f"\nAccuracy: {accuracy:.2%} ({correct}/{total})")

  print(f"Saved results to {results_path}")
  print(f"Saved summary to {summary_path}")


In [2]:
# !pip install -q transformers torch accelerate sentencepiece
!pip install -U datasets

In [3]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

MODEL_NAME = "Qwen/Qwen2-0.5B-Instruct"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LENGTH_GENERATION = 200 # Max tokens for rationale + answer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id


#def calculate_reward(final_answer, correct_answer):
#    """Calculates a binary reward based on correctness."""
#    if final_answer and final_answer == correct_answer:
#        return 1.0
#    return 0.0

/home/sanjay/miniconda/envs/224r/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class TaskEvaluator:
    def parse_llm_output(self, output: str):
        raise NotImplementedError

    def format_question(self, question_data: dict):
        raise NotImplementedError

    def format_prompt(self, question: str):
        raise NotImplementedError

In [5]:
import requests, re

class CommonsenseQAEval(TaskEvaluator):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.prompt_eos = self._load_prompt()

    def _load_prompt(self):
        url = "https://raw.githubusercontent.com/ezelikman/STaR/main/commonsenseqa/prompts.txt"
        prompt = requests.get(url).text
        prompt_eos = re.sub(r"\.\n\n", "." + self.tokenizer.eos_token + "\n\n", prompt)
        if not prompt.endswith(self.tokenizer.eos_token):
            prompt_eos = prompt_eos.rstrip() + self.tokenizer.eos_token
        return prompt_eos

    def format_question(self, question_data):
        question = question_data['question']
        choices_text = ""
        for i, choice_label in enumerate(question_data['choices']['label']):
            choice_text = question_data['choices']['text'][i]
            choices_text += f"({choice_label.lower()}) {choice_text}\n"

        return f"Q: {question}\nAnswer Choices:\n{choices_text.strip()}\nA: "

    def format_prompt(self, question_data):
        return f"{self.prompt_eos}\n\n{self.format_question(question_data)}The answer"

    def parse_llm_output(self, generated_text):
        rationale = generated_text
        final_answer = None

        final_answer = re.findall(r"\(([a-e])\)", generated_text, re.IGNORECASE)
        if final_answer:
            final_answer = final_answer[-1].lower()
            if len(final_answer) > 0:
                last_occurrence_index = generated_text.rfind(f"({final_answer})")
                if last_occurrence_index != -1:
                    rationale = generated_text[:last_occurrence_index].strip()
        return rationale, final_answer

In [6]:
from tqdm import tqdm


import torch.nn.functional as F
from tqdm import tqdm

def generate_responses_generic(
    dataset,
    model,
    tokenizer,
    evaluator,
    device,
    max_tokens=200,
    reward_fn=None,
    verbose=True,
):
    """
    Generic function that setups up eval
    """
    results = []
    model.to(device)
    model.eval()
    
    print(f"The dataset has {len(dataset)} examples\n")

    for idx, item in enumerate(tqdm(dataset)):
        correct_answer = item.get('answerKey', '').lower()
        print(f"\nThis is example {idx + 1}\n")

        # Format input prompt using evaluator
        prompt = evaluator.format_prompt(item)
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)

        # Generate model output
        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                temperature=0.7,
                top_k=20,
                top_p=0.8,
            )

        # Extract generated text
        n_input_tokens = inputs.input_ids.shape[1]
        generated_ids = output_ids[0, n_input_tokens:]
        generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

        # Compute log-probability of generated tokens
        logits = model(output_ids).logits[0, n_input_tokens:]
        log_probs_all = F.log_softmax(logits, dim=-1)
        token_log_probs = torch.gather(log_probs_all, 1, generated_ids.unsqueeze(1)).squeeze(1)
        total_log_prob = token_log_probs.sum().item()

        # Parse output
        rationale, final_answer = evaluator.parse_llm_output(generated_text)

        # Compute reward
        reward = reward_fn(final_answer, correct_answer) if reward_fn else None

        if verbose:
            print("\n--------------------")
            print(f"QUESTION:\n{evaluator.format_question(item)}")
            print(f"PROMPT:\n{prompt}")
            print(f"\nGENERATION:\n{generated_text}")
            print(f"\nPREDICTION: {final_answer}, TRUE ANSWER: {correct_answer}, REWARD: {reward}")
            print(f"logP(y | x) = {total_log_prob:.3f}")

        results.append({
            "question": evaluator.format_question(item),
            "prompt": prompt,
            "generated_text": generated_text,
            "final_answer": final_answer,
            "correct_answer": correct_answer,
            "rationale": rationale,
            "reward": reward,
            "logprob": total_log_prob,
        })

    return results


In [7]:
import random

commonsense_qa_dataset = load_dataset("commonsense_qa", split=f'validation')

# subsample 100 examples to evaluate in the test split
commonsense_qa_dataset = commonsense_qa_dataset.select(random.sample(range(len(commonsense_qa_dataset)), 3))


import requests, re
url = "https://raw.githubusercontent.com/ezelikman/STaR/main/commonsenseqa/prompts.txt"
prompt = requests.get(url).text
prompt_eos = re.sub(r"\.\n\n", "." + tokenizer.eos_token + "\n\n", prompt)
if not prompt.endswith(tokenizer.eos_token):
    prompt_eos = prompt_eos.rstrip() +  tokenizer.eos_token

evaluator = CommonsenseQAEval(tokenizer)

def reward_fn(pred, gold):
    return int(pred == gold)

results = generate_responses_generic(
    dataset=commonsense_qa_dataset,
    model=model,
    tokenizer=tokenizer,
    evaluator=evaluator,
    device=DEVICE,
    reward_fn=reward_fn,
)

logging(results, "commonsense_qa")

Generating test split: 100%|██████████| 1140/1140 [00:00<00:00, 43981.22 examples/s]


The dataset has 3 examples



  0%|          | 0/3 [00:00<?, ?it/s]


This is example 1



 33%|███▎      | 1/3 [00:01<00:02,  1.21s/it]


--------------------
QUESTION:
Q: Where is the large area location of the empire state building?
Answer Choices:
(a) manhattan
(b) office
(c) the city
(d) fifth avenue
(e) new york city
A: 
PROMPT:
Q: What do people use to absorb extra ink from a fountain pen?
Answer Choices:
(a) shirt pocket
(b) calligrapher's hand
(c) inkwell
(d) desk drawer
(e) blotter
A: The answer must be used to absorb extra ink. Blotters are designed to absorb liquids. Therefore, the answer is blotter (e).<|im_end|>

Q: What home entertainment equipment requires cable?
Answer Choices:
(a) radio shack
(b) substation
(c) television
(d) cabinet
(e) desk
A: The answer must require cable. Cable is used to provide satellite channels to televisions. Therefore, the answer is television (c).<|im_end|>

Q: The fox walked from the city into the forest, what was it looking for?
Answer Choices:
(a) pretty flowers
(b) hen house
(c) natural habitat
(d) storybook
(e) dense forest
A: The answer must be a reason for a fox to go 

 67%|██████▋   | 2/3 [00:01<00:00,  1.15it/s]


--------------------
QUESTION:
Q: What is a wet person likely to do?
Answer Choices:
(a) gain weight
(b) thank god
(c) catch cold
(d) suicide
(e) cross street
A: 
PROMPT:
Q: What do people use to absorb extra ink from a fountain pen?
Answer Choices:
(a) shirt pocket
(b) calligrapher's hand
(c) inkwell
(d) desk drawer
(e) blotter
A: The answer must be used to absorb extra ink. Blotters are designed to absorb liquids. Therefore, the answer is blotter (e).<|im_end|>

Q: What home entertainment equipment requires cable?
Answer Choices:
(a) radio shack
(b) substation
(c) television
(d) cabinet
(e) desk
A: The answer must require cable. Cable is used to provide satellite channels to televisions. Therefore, the answer is television (c).<|im_end|>

Q: The fox walked from the city into the forest, what was it looking for?
Answer Choices:
(a) pretty flowers
(b) hen house
(c) natural habitat
(d) storybook
(e) dense forest
A: The answer must be a reason for a fox to go into the forest. The forest

100%|██████████| 3/3 [00:02<00:00,  1.22it/s]


--------------------
QUESTION:
Q: What furniture will you normally find near a side chair?
Answer Choices:
(a) bedroom
(b) table
(c) wheel barrow
(d) building
(e) office
A: 
PROMPT:
Q: What do people use to absorb extra ink from a fountain pen?
Answer Choices:
(a) shirt pocket
(b) calligrapher's hand
(c) inkwell
(d) desk drawer
(e) blotter
A: The answer must be used to absorb extra ink. Blotters are designed to absorb liquids. Therefore, the answer is blotter (e).<|im_end|>

Q: What home entertainment equipment requires cable?
Answer Choices:
(a) radio shack
(b) substation
(c) television
(d) cabinet
(e) desk
A: The answer must require cable. Cable is used to provide satellite channels to televisions. Therefore, the answer is television (c).<|im_end|>

Q: The fox walked from the city into the forest, what was it looking for?
Answer Choices:
(a) pretty flowers
(b) hen house
(c) natural habitat
(d) storybook
(e) dense forest
A: The answer must be a reason for a fox to go into the forest.

In [8]:
# some of this processing code was written with assistance / consultation from an LLM for cookie cutter pre-processing logic for a CSV.

import csv
import re

def strip_html(text):
    """Remove HTML tags like <span class='from'> and <span class='to'>."""
    return re.sub(r"<[^>]+>", "", text)

def load_turk_data(file_path):
    examples = []

    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        for row in reader:
            # Collect training input/output pairs
            train_pairs = []
            for i in range(8):
                before = strip_html(row[f'before_{i}'])
                after = strip_html(row[f'after_{i}'])
                train_pairs.append((before, after))

            # Collect test input/output
            test_before = strip_html(row['test_before'])
            test_after = strip_html(row['test_after'])

            # Collect hint if available
            hint = strip_html(row['hint']).strip()

            # Store everything
            examples.append({
                "example_pairs": train_pairs,
                "test_query": test_before,
                "answerKey": test_after,
                "hint": hint if hint else None
            })

    return examples



In [9]:
first_set = load_turk_data('./turk_data.csv')
second_set = load_turk_data('./turk_data_2.csv')

string_editing_dataset = first_set + second_set
print(f"Loaded {len(string_editing_dataset)} examples.")

Loaded 2000 examples.


In [10]:
import random

sampled = random.sample(string_editing_dataset, 10)
for i, ex in enumerate(sampled, 1):
    print(f"\n--- Example {i} ---")
    print("Hint:", ex['hint'])
    print("Example pairs:")
    for before, after in ex['example_pairs']:
        print(f"  {before} → {after}")
    print("Test pair:")
    print(f"  {ex['example_pairs'][0]} → {ex['answerKey']}")


--- Example 1 ---
Hint: look at the beginning of the word
Example pairs:
  virago → vlirago
  spiting → slpiting
  elephantine → ellephantine
  nonrefillable → nlonrefillable
  migrant → mligrant
  paintwork → plaintwork
  madly → mladly
  fingerprinting → flingerprinting
Test pair:
  ('virago', 'vlirago') → alxiomatically

--- Example 2 ---
Hint: look at the end of the wordlook at consonants
Example pairs:
  diaper → diaper
  bouffants → bouffanlg
  varnishing → varnishilg
  torus → torus
  sebaceous → sebaceous
  sling → slilg
  explore → explolg
  ivy → ilg
Test pair:
  ('diaper', 'diaper') → sprulg

--- Example 3 ---
Hint: look at consonants
Example pairs:
  deciding → decidiy
  sticklebacks → yiylebays
  encouraged → eyouraged
  qualify → qualiy
  truncation → yuyation
  ionizes → ionizes
  quip → quip
  preppy → yeyy
Test pair:
  ('deciding', 'decidiy') → unayoyanied

--- Example 4 ---
Hint: look at the beginning of the wordlook at vowelslook at consonants
Example pairs:
  forei

In [ ]:
from typing import Optional
import random

class StringEditingEval(TaskEvaluator):
    def __init__(self, tokenizer, include_hint: bool = False):
        self.tokenizer = tokenizer
        self.include_hint = include_hint
        # Load from turk csvs
        #first_set = load_turk_data('./turk_data.csv')
        #second_set = load_turk_data('./turk_data_2.csv')
        #self.string_dataset = first_set + second_set
        self.prompt_eos = self.tokenizer.eos_token #self._load_prompt()

    #def _load_prompt(self):
    #    url = "https://raw.githubusercontent.com/ezelikman/STaR/main/commonsenseqa/prompts.txt"
    #    prompt = requests.get(url).text
    #    prompt_eos = re.sub(r"\.\n\n", "." + self.tokenizer.eos_token + "\n\n", prompt)
    #    if not prompt.endswith(self.tokenizer.eos_token):
    #        prompt_eos = prompt_eos.rstrip() + self.tokenizer.eos_token
    #    return prompt_eos

    def format_prompt(self, example):
        """Construct the prompt including optional hint."""
        examples = "\n".join([f"{b} -> {a}" for b, a in example["example_pairs"]])
        hint = f"\nHint: {example['hint'].rstrip()}" if self.include_hint and example.get("hint") else ""
        string_instruction=f"""Find the rule applied to edit these example pairs. Apply this rule to the word {example['test_query'].rstrip()} and return the answer as a single word. Think carefully and only return a single word."""
        return f"\n{examples}\n{hint}\n{string_instruction}\nAnswer:"

    def format_question(self, example):
        """Not needed for string editing."""
        return None

    def parse_llm_output(self, generated_text):
        """
        Return full output as rationale and the first token/string after 'Answer:' as final answer.
        """
        rationale = generated_text.strip()
        # Assume the answer is the first non-empty line of the output
        lines = [line.strip() for line in generated_text.strip().split('\n') if line.strip()]
        final_answer = lines[0] if lines else None
        return rationale, final_answer


In [32]:
# Run eval on String Editing Dataset
import numpy as np

evaluator = StringEditingEval(tokenizer)
base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)

SEED = 21
random.seed(SEED)
torch.manual_seed(SEED)

# subsample k examples to evaluate in the test split
k = 10
subset_indices = random.sample(range(len(string_editing_dataset)), k)
sample_string_editing = [string_editing_dataset[i] for i in subset_indices]

def reward_fn(pred, gold):
    return int(pred == gold)

results = generate_responses_generic(
    dataset=sample_string_editing,
    model=base_model,
    tokenizer=tokenizer,
    evaluator=evaluator,
    device=DEVICE,
    reward_fn=reward_fn,
)

logging(results, "string_editing")

The dataset has 10 examples



  0%|          | 0/10 [00:00<?, ?it/s]


This is example 1



 10%|█         | 1/10 [00:01<00:11,  1.33s/it]


--------------------
QUESTION:
None
PROMPT:

winding -> wydyg
avengers -> yygys
unluckier -> ylykyr
shut -> shy
ramrodding -> ryrydyg
blamelessly -> blyyysly
anchored -> ychyy
razzed -> ryzy

twinging -> ?

GENERATION:
?
tired -> ??
barking -> ??
sitting -> ??

The output should be the same as the input, but with some words changed to their more appropriate forms. The change should not alter the meaning of the original sentence.
I'm sorry, but I can't assist with that.

Please provide a list of words and phrases you'd like me to change so they fit in the given context. Additionally, please specify if any of the changes are acceptable or not, and which ones would work best for each word.

PREDICTION: ?, TRUE ANSWER: twygyg, REWARD: 0
logP(y | x) = -1416.758

This is example 2



...


--------------------
QUESTION:
None
PROMPT:

filter -> fbiblbtbebrb
eastbound -> ebabsbtbbbobubnbdb
promulgates -> pbrbobmbublbgbabtbebsb
burnish -> bbubrbnbibsbhb
quadrilles -> qbubabdbrbiblblbebsb
anomalies -> abnbobmbablbibebsb
exit -> ebxbibtb
chunkier -> cbhbubnbkbibebrb

dentin -> ?

GENERATION:
bbibttbbubnbc
fossils -> bcbibttbbubnbcb
excremental -> bcbibttbbubnbcb
disruption -> bcbibttbbubnbcb
crustless -> bcbibttbbubnbcb
stainless -> bcbibttbbubnbcb
streaky -> bcbibttbbubnbcb
sweatless -> bcbibttbbubnbcb
stainless -> bcbibttbbubnbcb
wetnessless -> bcbibttbbubnbcb
stainless -> bcbibttbbubnbcb
drynessless -> bcbibttbbubnbcb
wetnessless -> bcbibttbbubnbcb
sweatless -> bcbibttbbubnbcb
stainless -> bcbibttbbubnbcb
drynessless -> bcbib

PREDICTION: bbibttbbubnbc, TRUE ANSWER: dbebnbtbibnb, REWARD: 0
logP(y | x) = -2426.699

This is example 9



 90%|█████████ | 9/10 [00:18<00:02,  2.21s/it]


--------------------
QUESTION:
None
PROMPT:

offices -> tfices
umber -> tber
export -> tport
stagnating -> stagnating
clew -> clew
pediatrist -> pediatrist
decentralized -> decentralized
inflatables -> tflatables

indorse -> ?

GENERATION:
?
solarize -> solarize
catalyze -> catalyze
dissolve -> dissolve
simplify -> simplify
disassemble -> disassemble
resemble -> resemble
tackle -> tackle
simplify -> simplify
sustain -> sustain
substitute -> substitute
transmit -> transmit
transform -> transform
unify -> unify
undistribute -> undistribute
unite -> unite
unify -> unify
understand -> understand
verify -> verify
validate -> validate
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work -> work
work ->

PREDICTION: ?, TRUE ANSWER: tdorse, REWAR

100%|██████████| 10/10 [00:21<00:00,  2.12s/it]


--------------------
QUESTION:
None
PROMPT:

nervelessly -> cnecrveclecsscly
trouser -> ctroucser
traps -> ctracps
buccaneered -> cbucccacneecred
swanks -> cswacnks
sparring -> cspacrricng
tabs -> ctacbs
homeboy -> chocmecboy

hick -> ?

GENERATION:
hick
toddler -> ?todler
dude -> ?dude
guy -> ?guy
girl -> ?girl
fem -> ?fem
male -> ?male
female -> ?female
dog -> ?dog
cat -> ?cat
horse -> ?horse
pig -> ?pig
cow -> ?cow
duck -> ?duck
catwoman -> ?catwomann
dogman -> ?dogman
bull -> ?bull
bear -> ?bear
rabbit -> ?rabbit
sheep -> ?sheep
fox -> ?fox
deer -> ?deer
goat -> ?goat
chicken -> ?chicken
penguin -> ?penguin
zebra -> ?zebra
kangaroo -> ?kangaroo
snake -> ?snake
crocodile -> ?crocodile
elephant -> ?elephant
lion -> ?lion
tiger -> ?tiger
koala -> ?koala


PREDICTION: hick, TRUE ANSWER: chicck, REWARD: 0
logP(y | x) = -2818.938

Accuracy: 0.00% (0/10)
Saved results to outputs/string_editing_results_2.json
Saved summary to outputs/string_editing_summary_2.json


In [30]:
from tqdm.auto import tqdm 
import torch.nn.functional as F

def lm_score_rationale(model, tokenizer, item, generated_text, device, max_tokens=10):
    """
    Evaluate a generated rationale via two reward components:
    1. A scalar score from the LM judging rationale quality (0 to 1). Rationale_score: float between [0, 1] from LM
    2. A binary reward based on whether the final predicted answer matches the ground truth (0 or 1). Correctness_reward: float between {0.0, 1.0}

    Returns:
        Combined score equally weighted
    """
    rationale, answer = evaluator.parse_llm_output(generated_text)
    question = item["question"]
    gold_answer = item["answerKey"]

    # Construct evaluation prompt
    #eval_prompt = (
    #    f"{question}\n{rationale}\n{answer}\n"
    #    "Return a numerical score to this rationale between 0 (bad) and 1 (perfect), "
    #    "evaluating the rationale based on how well it answers the original question, "
    #    "the quality of its reasoning, its logical coherence and consistency, and importantly, "
    #    "whether the final answer was correct. Just output the decimal score between 0 and 1, to the nearest tenth. for example: 0.1."""
    #)
    eval_prompt = (
        f"{question}\n"
        f"{rationale}\n"
        f"Answer: {answer}\n"
        f"\nEvaluate the quality of the rationale above. Only respond with a number between 0 and 1, where 0 is a poor rationale and 1 is a perfect one.\nScore:"
    )

    #print(eval_prompt)
    # Tokenize input
    inputs = tokenizer(eval_prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)

    # Generate score prediction from the LM
    with torch.no_grad():
        # Greedy decode
        output = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False,
        )

    generated_score_text = tokenizer.decode(output[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True).strip()
    match = re.search(r"\b([01](?:\.\d+)?|0?\.\d+)\b", generated_score_text)

    # Extract first float from the response
    if match:
        lm_score = float(match.group(1)) 
        lm_score = max(0.0, min(1.0, lm_score))
    else:
        lm_score = 0.0
    
    #print(f"LM SCORE: {lm_score}")
    
    # If answer failed to parse, don't answer
    answer = "" if isinstance(answer, list) else answer
    correctness_reward = float(answer.strip().lower() == gold_answer.strip().lower())
    score = 0.5 * correctness_reward + 0.5 * lm_score
    return score

def get_optimizer(model, lr=3.5e-3):
  return torch.optim.AdamW(model.parameters(), lr=1e-5)

def train_with_rationale_rewards(dataset, model, tokenizer, evaluator, device, epochs=5, lr=1e-4):
    """
    Train arbitrary model with process-level (i.e. fine-grained) reward signal 
    """
    model.train()
    optimizer = get_optimizer(model, lr=lr)

    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}")
        for item in tqdm(dataset, desc="Training"):
            # Step 1: Sample (rationale, answer)
            prompt = evaluator.format_prompt(item)
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
            
            output_ids = model.generate(
                **inputs,
                max_new_tokens=MAX_LENGTH_GENERATION,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                temperature=0.7,
                top_k=20,
                top_p=0.8,
            )

            n_input_tokens = inputs.input_ids.shape[1]
            generated_ids = output_ids[0, n_input_tokens:]
            generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
            rationale, final_answer = evaluator.parse_llm_output(generated_text)

            # Step 2: Evaluate reward
            reward = lm_score_rationale(model, tokenizer, item, generated_text, device)

            # Step 3: Compute log-probs
            logits = model(output_ids).logits[0, n_input_tokens:]
            log_probs_all = F.log_softmax(logits, dim=-1)
            token_log_probs = torch.gather(log_probs_all, 1, generated_ids.unsqueeze(1)).squeeze(1)
            total_log_prob = token_log_probs.sum()

            # Step 4: Compute policy gradient loss
            loss = -reward * total_log_prob

            # Step 5: Update policy
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()

            if hasattr(torch.cuda, "empty_cache"):
                torch.cuda.empty_cache()

In [25]:
# Train base qwen model with fine-grained rewards on string editing
import random
evaluator = StringEditingEval(tokenizer)
train_string_editing = random.sample(string_editing_dataset, k=500)
train_with_rationale_rewards(train_string_editing, model, tokenizer, evaluator, DEVICE, epochs=1)


Epoch 1


Training: 100%|██████████| 500/500 [20:31<00:00,  2.46s/it]


In [27]:
test_string_editing = [x for x in string_editing_dataset if x not in train_string_editing]

def reward_fn(pred, gold):
    return int(pred == gold)

# Randomly sample 100 test examples
results = generate_responses_generic(
    dataset=random.sample(test_string_editing, k=100),
    model=model,
    tokenizer=tokenizer,
    evaluator=evaluator,
    device=DEVICE,
    reward_fn=reward_fn,
    verbose=True
)


The dataset has 100 examples



  0%|          | 0/100 [00:00<?, ?it/s]


This is example 1



  1%|          | 1/100 [00:02<03:55,  2.38s/it]


--------------------
QUESTION:
None
PROMPT:

mike is edited to mhkh<|im_end|>
cryptographers is edited to crypthgrhphhrs<|im_end|>
spotting is edited to sphtthng<|im_end|>
benign is edited to bhnhgn<|im_end|>
marmosets is edited to mhrmhshts<|im_end|>
bedrooms is edited to bhdrhhms<|im_end|>
fixating is edited to fhxhthng<|im_end|>
profile is edited to prhfhlh<|im_end|>

Find the rule applied to edit these example pairs. Apply this rule to the word medleys and return the answer as a single word. Think carefully and only return a single word.
Answer:

GENERATION:
.)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)


...


--------------------
QUESTION:
None
PROMPT:

pleasantry is edited to jpjleajsajnjtjrjy<|im_end|>
denuding is edited to jdejnujdijnjg<|im_end|>
rains is edited to jraijnjs<|im_end|>
smash is edited to jsjmajsjh<|im_end|>
lumped is edited to jlujmjpejd<|im_end|>
gulch is edited to jgujljcjh<|im_end|>
cigarets is edited to jcijgajrejtjs<|im_end|>
endowing is edited to ejnjdojwijnjg<|im_end|>

Find the rule applied to edit these example pairs. Apply this rule to the word wristwatches and return the answer as a single word. Think carefully and only return a single word.
Answer:

GENERATION:
.)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 

 99%|█████████▉| 99/100 [03:53<00:02,  2.33s/it]


--------------------
QUESTION:
None
PROMPT:

foregoes is edited to xnxnxnxnxnxnxnxn<|im_end|>
mozzarella is edited to xnxnxnxnxnxnxnxnxnxn<|im_end|>
forebode is edited to xnxnxnxnxnxnxnxn<|im_end|>
bases is edited to xnxnxnxnxn<|im_end|>
naughtier is edited to xnxnxnxnxnxnxnxnxn<|im_end|>
insisted is edited to xnxnxnxnxnxnxnxn<|im_end|>
unclothe is edited to xnxnxnxnxnxnxnxn<|im_end|>
anonymously is edited to xnxnxnxnxnxnxnxnxnxnxn<|im_end|>

Find the rule applied to edit these example pairs. Apply this rule to the word granny and return the answer as a single word. Think carefully and only return a single word.
Answer:

GENERATION:
.)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .

100%|██████████| 100/100 [03:56<00:00,  2.36s/it]


--------------------
QUESTION:
None
PROMPT:

despatching is edited to ldlelslplaltlclhlilnlg<|im_end|>
lifting is edited to lllilfltlilnlg<|im_end|>
snows is edited to lslnlolwls<|im_end|>
endeavoring is edited to lelnldlelalvlolrlilnlg<|im_end|>
mahjong is edited to lmlalhljlolnlg<|im_end|>
bewildered is edited to lblelwlillldlelrleld<|im_end|>
crasser is edited to lclrlalslslelr<|im_end|>
discover is edited to ldlilslclolvlelr<|im_end|>

Find the rule applied to edit these example pairs. Apply this rule to the word sheller and return the answer as a single word. Think carefully and only return a single word.
Answer:

GENERATION:
.)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)



 .)


In [31]:
# Train base qwen model with fine-grained rewards on common senseqa
import random
commonsense_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)
commonsense_qa_dataset_train = load_dataset("commonsense_qa", split=f'train')
evaluator = CommonsenseQAEval(tokenizer)
train_commonsense = commonsense_qa_dataset_train.select(random.sample(range(len(commonsense_qa_dataset_train)), k=500))
train_with_rationale_rewards(train_commonsense, commonsense_model, tokenizer, evaluator, DEVICE, epochs=1)


Epoch 1


Training:  36%|███▋      | 182/500 [07:38<13:21,  2.52s/it]


KeyboardInterrupt: 

In [32]:
commonsense_qa_dataset_test = load_dataset("commonsense_qa", split=f'validation')

def reward_fn(pred, gold):
    return int(pred == gold)

# Randomly sample 100 test examples
results = generate_responses_generic(
    dataset=commonsense_qa_dataset_test.select(random.sample(range(len(commonsense_qa_dataset_test)), k=100)),
    model=commonsense_model,
    tokenizer=tokenizer,
    evaluator=evaluator,
    device=DEVICE,
    reward_fn=reward_fn,
    verbose=False
)


The dataset has 100 examples



  0%|          | 0/100 [00:00<?, ?it/s]


This is example 1



  1%|          | 1/100 [00:02<03:54,  2.37s/it]


This is example 2



...

 98%|█████████▊| 98/100 [03:53<00:04,  2.38s/it]


This is example 99



 99%|█████████▉| 99/100 [03:55<00:02,  2.38s/it]


This is example 100



100%|██████████| 100/100 [03:58<00:00,  2.38s/it]


In [33]:
logging(results, "commonsense_qa_rl")


Accuracy: 0.00% (0/100)
Saved results to outputs/commonsense_qa_rl_results_4.json
Saved summary to outputs/commonsense_qa_rl_summary_4.json
